In [1]:
library(Seurat)
library(reticulate)

The legacy packages maptools, rgdal, and rgeos, underpinning this package
will retire shortly. Please refer to R-spatial evolution reports on
https://r-spatial.org/r/2023/05/15/evolution4.html for details.
This package is now running under evolution status 0 

Attaching SeuratObject



In [2]:
use_python("/data/chenyx/anaconda3/envs/PICASSO/bin")

In [3]:
sc <- import("scanpy")
sp <- import("scipy.sparse")

# load data

In [ ]:
adata <- sc$read_h5ad("/nfs/public/cell_gpt_data/Intergation_COVID/raw_uniformed.h5ad")

In [3]:
expression_matrix = as.matrix(adata$X)
rownames(expression_matrix) <- row.names(adata$obs)
colnames(expression_matrix) <- row.names(adata$var)

Warning message in asMethod(object):
“sparse->dense coercion: allocating vector of size 86.1 GiB”


In [4]:
dataobj <- CreateSeuratObject(counts = t(expression_matrix), meta.data = adata$obs, min.cells = 0,min.features = 0)
dataobj

Warning message:
“Invalid name supplied, making object name syntactically valid. New object name is ageage_rangeanatomical_regionanatomical_region_detailedbatchdatasetdiseasedonorethnicityethnicity_mixedlast_author.PIlung_vs_nasaloriginal_celltype_annpack_yearssamplesample_aliassample_typesampling_methodsexsmokingtotal_countslog10_total_countsn_genes_detectedmito_fracribo_fraccomplann_level_1ann_level_2ann_level_3ann_level_4ann_level_5ann_highest_resann_newsubject_typestudystudy2celltypeconditioncellnamesclusterstageIDsample_newchemistrydata_typedpt_pseudotimefinal_annotationmt_fracn_countsn_genessample_IDsize_factorsspeciestissue; see ?make.names for more details on syntax validity”


An object of class Seurat 
42117 features across 274346 samples within 1 assay 
Active assay: RNA (42117 features, 0 variable features)

In [5]:
dataobj <- NormalizeData(dataobj)

In [7]:
dataobj <- dataobj[read.csv("gene_list.csv")$gene_name,]

In [8]:
dataobj

An object of class Seurat 
2000 features across 274346 samples within 1 assay 
Active assay: RNA (2000 features, 0 variable features)

In [9]:
saveRDS(dataobj, "/nfs//public/cell_gpt_data/Intergation_COVID/Results/selected_gene.rds")

# FastMNN

In [4]:
library(SeuratWrappers)

In [5]:
dataobj <- readRDS("/nfs//public/cell_gpt_data/Intergation_COVID/Results/selected_gene.rds")

In [6]:
start_time = Sys.time()
dataobj.fastmnn <- RunFastMNN(object.list = SplitObject(dataobj, split.by = "study"))
end_time = Sys.time()

Computing 2000 integration features

No variable features found for object1 in the object.list. Running FindVariableFeatures ...

No variable features found for object2 in the object.list. Running FindVariableFeatures ...

No variable features found for object3 in the object.list. Running FindVariableFeatures ...

No variable features found for object4 in the object.list. Running FindVariableFeatures ...

No variable features found for object5 in the object.list. Running FindVariableFeatures ...

No variable features found for object6 in the object.list. Running FindVariableFeatures ...

No variable features found for object7 in the object.list. Running FindVariableFeatures ...

No variable features found for object8 in the object.list. Running FindVariableFeatures ...

No variable features found for object9 in the object.list. Running FindVariableFeatures ...

No variable features found for object10 in the object.list. Running FindVariableFeatures ...

No variable features found for o

In [7]:
end_time- start_time

Time difference of 23.81625 mins

In [19]:
saveRDS(dataobj.fastmnn, "/nfs//public/cell_gpt_data/Intergation_COVID/Results/FastMNN_result.rds")

In [20]:
dataobj.fastmnn@active.assay <- "mnn.reconstructed"

In [21]:
dataobj.fastmnn

An object of class Seurat 
3961 features across 274346 samples within 2 assays 
Active assay: mnn.reconstructed (1961 features, 1961 variable features)
 1 other assay present: RNA
 1 dimensional reduction calculated: mnn

In [22]:
X = t(as.matrix(dataobj.fastmnn@assays$mnn.reconstructed@data))
var = VariableFeatures(dataobj.fastmnn)
obs = dataobj.fastmnn[[]]

Warning message in asMethod(object):
“sparse->dense coercion: allocating vector of size 4.0 GiB”


In [23]:
adata_seurat <- sc$AnnData(
    X   = X,
    obs = obs,
    var = VariableFeatures(dataobj.fastmnn)
)
adata_seurat$X  <- sp$csc_matrix(adata_seurat$X)

In [24]:
colnames(adata_seurat$var) <- "gene_name"
adata_seurat

AnnData object with n_obs × n_vars = 274346 × 1961
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'age', 'age_range', 'anatomical_region', 'anatomical_region_detailed', 'batch', 'dataset', 'disease', 'donor', 'ethnicity', 'ethnicity_mixed', 'last_author.PI', 'lung_vs_nasal', 'original_celltype_ann', 'pack_years', 'sample', 'sample_alias', 'sample_type', 'sampling_method', 'sex', 'smoking', 'total_counts', 'log10_total_counts', 'n_genes_detected', 'mito_frac', 'ribo_frac', 'compl', 'ann_level_1', 'ann_level_2', 'ann_level_3', 'ann_level_4', 'ann_level_5', 'ann_highest_res', 'ann_new', 'subject_type', 'study', 'study2', 'celltype', 'condition', 'cellnames', 'cluster', 'stage', 'ID', 'sample_new', 'chemistry', 'data_type', 'dpt_pseudotime', 'final_annotation', 'mt_frac', 'n_counts', 'n_genes', 'sample_ID', 'size_factors', 'species', 'tissue'
    var: 'gene_name'

In [25]:
adata_seurat$write_h5ad("/nfs//public/cell_gpt_data/Intergation_COVID/Results/FastMNN_result.h5ad")

# Seurat integration

In [26]:
dataobj.list <- SplitObject(dataobj, split.by = "study")

In [ ]:
anchors <- FindIntegrationAnchors(object.list = dataobj.list, anchor.features = read.csv("gene_list.csv")$gene_name)

Scaling features for provided objects

Finding all pairwise anchors

Running CCA

Merging objects

Finding neighborhoods

Finding anchors

	Found 13184 anchors

Filtering anchors

	Retained 3957 anchors

Running CCA

Merging objects

Finding neighborhoods

Finding anchors

	Found 14417 anchors

Filtering anchors

	Retained 4643 anchors

Running CCA

Merging objects

Finding neighborhoods

Finding anchors

	Found 16907 anchors

Filtering anchors

	Retained 5316 anchors

Running CCA

Merging objects

Finding neighborhoods

Finding anchors

	Found 8137 anchors

Filtering anchors

	Retained 4128 anchors

Running CCA

Merging objects

Finding neighborhoods

Finding anchors

	Found 8930 anchors

Filtering anchors

	Retained 4722 anchors

Running CCA

Merging objects

Finding neighborhoods

Finding anchors

	Found 11869 anchors

Filtering anchors

	Retained 7079 anchors

Running CCA

Merging objects

Finding neighborhoods

Finding anchors

	Found 10800 anchors

Filtering anchors

	Retained 41

In [36]:
dataobj.integrated <- IntegrateData(anchorset = anchors)

Merging dataset 15 into 14

Extracting anchors for merged samples

Finding integration vectors

Finding integration vector weights

Integrating data

Merging dataset 18 into 7

Extracting anchors for merged samples

Finding integration vectors

Finding integration vector weights

Integrating data

Merging dataset 10 into 8

Extracting anchors for merged samples

Finding integration vectors

Finding integration vector weights

Integrating data

Merging dataset 17 into 14 15

Extracting anchors for merged samples

Finding integration vectors

Finding integration vector weights

Integrating data

Merging dataset 5 into 2

Extracting anchors for merged samples

Finding integration vectors

Finding integration vector weights

Integrating data

Merging dataset 16 into 13

Extracting anchors for merged samples

Finding integration vectors

Finding integration vector weights

Integrating data

Merging dataset 12 into 8 10

Extracting anchors for merged samples

Finding integration vectors

Fin

In [37]:
dataobj.integrated

An object of class Seurat 
4000 features across 274346 samples within 2 assays 
Active assay: integrated (2000 features, 2000 variable features)
 1 other assay present: RNA

In [38]:
saveRDS(dataobj.integrated, "/nfs//public/cell_gpt_data/Intergation_COVID/Results/Seurat_result.rds")

In [39]:
X = t(as.matrix(dataobj.integrated@assays$integrated@data))
var = VariableFeatures(dataobj.integrated)
obs = dataobj.integrated[[]]

Warning message in asMethod(object):
“sparse->dense coercion: allocating vector of size 4.1 GiB”


In [40]:
adata_seurat <- sc$AnnData(
    X   = X,
    obs = obs,
    var = VariableFeatures(dataobj.integrated)
)
adata_seurat$X  <- sp$csc_matrix(adata_seurat$X)
colnames(adata_seurat$var) <- "gene_name"
adata_seurat

AnnData object with n_obs × n_vars = 274346 × 2000
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'age', 'age_range', 'anatomical_region', 'anatomical_region_detailed', 'batch', 'dataset', 'disease', 'donor', 'ethnicity', 'ethnicity_mixed', 'last_author.PI', 'lung_vs_nasal', 'original_celltype_ann', 'pack_years', 'sample', 'sample_alias', 'sample_type', 'sampling_method', 'sex', 'smoking', 'total_counts', 'log10_total_counts', 'n_genes_detected', 'mito_frac', 'ribo_frac', 'compl', 'ann_level_1', 'ann_level_2', 'ann_level_3', 'ann_level_4', 'ann_level_5', 'ann_highest_res', 'ann_new', 'subject_type', 'study', 'study2', 'celltype', 'condition', 'cellnames', 'cluster', 'stage', 'ID', 'sample_new', 'chemistry', 'data_type', 'dpt_pseudotime', 'final_annotation', 'mt_frac', 'n_counts', 'n_genes', 'sample_ID', 'size_factors', 'species', 'tissue'
    var: 'gene_name'

In [41]:
adata_seurat$write_h5ad("/nfs//public/cell_gpt_data/Intergation_COVID/Results/Seurat_result.h5ad")